In [1]:
import pandas as pd

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [48]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import io
import pathlib

In [71]:
from torch.utils.data.sampler import SubsetRandomSampler

In [6]:
skip_training = False

In [7]:
device = torch.device('cuda:0')
#device = torch.device('cpu')

In [8]:
if skip_training:
    # The models are always evaluated on CPU
    device = torch.device("cpu")

In [43]:
data_dir = '../data'

In [ ]:
train_set = datasets.ImageFolder("root/label/train", transform = transformations)

In [6]:
image_information = pd.read_csv("data/HAM10000_metadata.csv")

In [7]:
image_information

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear
5,HAM_0001466,ISIC_0027850,bkl,histo,75.0,male,ear
6,HAM_0002761,ISIC_0029176,bkl,histo,60.0,male,face
7,HAM_0002761,ISIC_0029068,bkl,histo,60.0,male,face
8,HAM_0005132,ISIC_0025837,bkl,histo,70.0,female,back
9,HAM_0005132,ISIC_0025209,bkl,histo,70.0,female,back


In [11]:
transform = transforms.Compose([
    transforms.ToTensor(),  # Transform to tensor
    transforms.Normalize((0.5,), (0.5,))  # Min-max scaling to [-1, 1]
])

In [91]:
#https://discuss.pytorch.org/t/custom-dataset-with-some-preprocessing/30385/2

class HAMN10000DataSet(Dataset):

    def __init__(self, txt_path, img_dir, transform=None):
    
        df = pd.read_csv(txt_path)
        self.img_dir = img_dir
        self.txt_path = txt_path
        self.img_names = df['image_id'] + '.jpg' #name image id
        self.y = df['dx'].values
        self.transform = transform

    def __getitem__(self, index):
        img = Image.open(os.path.join(self.img_dir,
                                      self.img_names[index]))
        
        if self.transform is not None:
            img = self.transform(img)
        
        label = self.y[index]
        return img, label

    def __len__(self):
        return self.y.shape[0]

In [94]:
dataset = HAMN10000DataSet(txt_path='data/HAM10000_metadata.csv',
                              img_dir='data/images',
                              transform=transform)

In [92]:
y = image_information.dx

X_train, X_val, y_train, y_test = train_test_split(y,y, test_size=0.2, stratify=y)
#np.unique(y_train, return_counts=True)
#np.unique(y_val, return_counts=True)

In [93]:
train_sampler = SubsetRandomSampler(y_train)
test_sampler = SubsetRandomSampler(y_test)

In [95]:
train_loader = torch.utils.data.DataLoader(dataset, batch_size=32, 
                                           sampler=train_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=32,
                                                sampler=test_sampler)

In [58]:
train_dataset = CelebaDataset(txt_path='data/HAM10000_metadata.csv',
                              img_dir='data/images',
                              transform=transform)

In [63]:
#train_dataset.__getitem__(0)
#train_dataset.img_names

In [64]:
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
#testloader = torch.utils.data.DataLoader(testset, batch_size=5, shuffle=False)

In [96]:
dataiter = iter(trainloader)
img, label = dataiter.next()
print(img[0].shape)

torch.Size([3, 450, 600])


In [66]:
img

tensor([[[[ 0.6863,  0.7020,  0.7020,  ...,  0.5922,  0.5608,  0.6000],
          [ 0.6863,  0.6627,  0.7333,  ...,  0.5765,  0.5373,  0.5608],
          [ 0.7412,  0.7020,  0.7725,  ...,  0.5686,  0.5608,  0.5529],
          ...,
          [ 0.5843,  0.6078,  0.6235,  ...,  0.5765,  0.5686,  0.5373],
          [ 0.6157,  0.6157,  0.5922,  ...,  0.6157,  0.6078,  0.5294],
          [ 0.6235,  0.6314,  0.6157,  ...,  0.5686,  0.6157,  0.5843]],

         [[ 0.5725,  0.5686,  0.5922,  ...,  0.5176,  0.4863,  0.5216],
          [ 0.5843,  0.5765,  0.6000,  ...,  0.5294,  0.5020,  0.5059],
          [ 0.5843,  0.5686,  0.6039,  ...,  0.5294,  0.5098,  0.4902],
          ...,
          [ 0.5647,  0.5608,  0.5725,  ...,  0.5333,  0.5647,  0.5490],
          [ 0.5569,  0.5529,  0.5647,  ...,  0.5647,  0.5647,  0.5333],
          [ 0.5412,  0.5569,  0.5686,  ...,  0.5412,  0.5647,  0.5569]],

         [[ 0.5843,  0.6118,  0.6078,  ...,  0.5412,  0.5020,  0.5451],
          [ 0.6000,  0.6000,  

In [36]:
train_dataset.__getitem__(0)

FileNotFoundError: [Errno 2] No such file or directory: './data/images/ISIC_0027419'

In [55]:
#img = Image.open(r'data/images/ISIC_0027419')
#img = Image.open(open("/data/images/ISIC_0027419", 'rb'))
img = Image.open('data/images/ISIC_0027419.jpg')

In [59]:
#img = Image.open(r'data/images/ISIC_0027419')

FileNotFoundError: [Errno 2] No such file or directory: 'data/images/ISIC_0027419'